# Simularium Conversion Tutorial : Custom Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio import Converter

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium JSON file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Shape your data

The Simularium custom data Converter consumes spatiotemporal data from any source. It requires the following data:
* **box_size** : *np.ndarray (shape = [3])*
    * A numpy ndarray containing the XYZ dimensions of the simulation bounding volume
* **times** : *np.ndarray (shape = [timesteps])*
    * A numpy ndarray containing the elapsed simulated time at each timestep
* **n_agents** : *np.ndarray (shape = [timesteps])*
    * A numpy ndarray containing the number of agents that exist at each timestep
* **viz_types** : *np.ndarray (shape = [timesteps, agents])*
    * A numpy ndarray containing the viz type for each agent at each timestep. Current options:
        * 1000 : default,
        * 1001 : fiber (which will require subpoints)
* **positions** : *np.ndarray (shape = [timesteps, agents, 3])*
    * A numpy ndarray containing the XYZ position for each agent at each timestep
* **types** : *List[List[str]] (list of shape [timesteps, agents])*
    * A list containing timesteps, for each a list of the string name for the type of each agent
* **radii** : *np.ndarray (shape = [timesteps, agents])*
    * A numpy ndarray containing the radius for each agent at each timestep
    
Optionally, you can also include:
* **subpoints** : *np.ndarray (shape = [timesteps, agents, subpoints, 3])*
    * A numpy ndarray containing a list of subpoint position data for each agent at each timestep. These values are currently only used for fiber agents.
* **plots** : *Dict[str, Any]*
    * An object containing plot data already in Simularium format

### Generate example data

To demonstrate using the custom converter, we'll first generate some random example data.

In [2]:
from string import ascii_uppercase
from random import choice

# parameters
total_steps = 5
timestep = 0.5
box_size = 100
n_agents = 5
min_radius = 5
max_radius = 10
agents_are_fibers = False
points_per_fiber = 4

example_data = {
    "box_size" : np.array([box_size, box_size, box_size]),
    "times" : timestep * np.array(list(range(total_steps))),
    "n_agents" : np.array(total_steps * [n_agents]),
    "types" : []
}
for t in range(total_steps):
    example_data["types"].append([choice(ascii_uppercase) for i in range(n_agents)])

### Sphere visualization

If the agents are particles to be visualized as spheres, their centroid positions can be added like this:

In [3]:
if not agents_are_fibers:
    example_data["positions"] = np.random.uniform(size=(total_steps, n_agents, 3)) * box_size - box_size * 0.5
    example_data["viz_types"] = np.array(total_steps * [n_agents * [1000.0]]) # default viz type = 1000
    example_data["radii"] = (max_radius - min_radius) * np.random.uniform(size=(total_steps, n_agents)) + min_radius

### Line visualization

If the agents are fibers to be visualized as lines, the positions of points along each fiber can be added using the optional subpoints:

In [4]:
if agents_are_fibers:
    example_data["positions"] = np.zeros(shape=(total_steps, n_agents, 3))
    example_data["subpoints"] = box_size * np.random.uniform(
        size=(total_steps, n_agents, points_per_fiber, 3)) - box_size * 0.5
    example_data["viz_types"] = np.array(total_steps * [n_agents * [1001.0]]) # fiber viz type = 1001
    example_data["radii"] = np.zeros(shape=(total_steps, n_agents))

Inspect the resulting data:

In [5]:
print("box size = {}".format(example_data["box_size"]))
print("times = {}".format(example_data["times"]))
print("number of agents = {}".format(example_data["n_agents"]))
print("viz types = {}".format(example_data["viz_types"]))
print("positions = {}".format(example_data["positions"]))
print("types = {}".format(example_data["types"]))
print("radii = {}".format(example_data["radii"]))
if agents_are_fibers:
    print("subpoints = {}".format(example_data["subpoints"]))

box size = [100 100 100]
times = [0.  0.5 1.  1.5 2. ]
number of agents = [5 5 5 5 5]
viz types = [[1000. 1000. 1000. 1000. 1000.]
 [1000. 1000. 1000. 1000. 1000.]
 [1000. 1000. 1000. 1000. 1000.]
 [1000. 1000. 1000. 1000. 1000.]
 [1000. 1000. 1000. 1000. 1000.]]
positions = [[[ 23.70772688 -32.56532781 -17.5531989 ]
  [ -2.79102546  41.1418211  -37.47892735]
  [ 43.05565384 -26.49435672  30.58426363]
  [-20.58970393  42.78752365   2.39556781]
  [  2.37001221  28.70675429 -48.15685904]]

 [[ 19.99804983  48.93656135 -14.26324433]
  [-31.93947942  13.83775032 -24.29599901]
  [ 29.58852541  -5.28747035  19.92551721]
  [ -1.39302951  20.0876287   32.482488  ]
  [-49.48565243  49.25921487   8.40248245]]

 [[-21.9528585   11.17734508  24.19171333]
  [ 37.36394936  22.44684807  12.12827158]
  [-21.31786528  49.75378135  20.16742979]
  [-36.56024478  16.07627889  -9.03140524]
  [-14.28612073 -13.69610298 -33.17266318]]

 [[ 19.90248336 -27.94087285  26.13298422]
  [ 18.84707561 -37.78252923  

## Convert and save as .simularium JSON file

Once your data is shaped like in the `example_data` object, you can use the converter to generate the file at the given path:

In [6]:
Converter(example_data).write_JSON("/Users/blairl/Desktop/example")

## Visualize in the Simularium viewer

In your browser, either Firefox or Chrome, navigate to https://staging.agentviz.allencell.org/ and drag your file onto the center viewer window. 

*For now you'll first have to choose an example trajectory and close the load window. Once the example trajectory loads, you can drop your own file in to replace it. We'll fix this soon :)*